In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from nltools.data import Brain_Data, Design_Matrix, Adjacency
import networkx as nx
from scipy import signal
from mne_connectivity import spectral_connectivity_epochs
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
from itertools import cycle
from matplotlib.lines import Line2D
from scipy.fft import fft, fftfreq
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from matplotlib.animation import FuncAnimation, PillowWriter
import plotly.graph_objs as go
from functools import reduce
import teneto
from teneto import TemporalNetwork
import json
from sklearn.metrics import roc_curve, auc
import numpy as np


from functions.Connectivity import *

In [ ]:
patient='sub-HUP185'
raw=mne.io.read_raw_fif(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_filtered.fif', preload=True)
#Reading a npy file 
data_alpha = np.load(f'//home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_alpha_aec_dense.npy')
data_beta = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_beta_aec_dense.npy')
data_hgamma = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_high_gamma1_aec_dense.npy')
data_lgamma = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_low_gamma_aec_dense.npy')
data_theta = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_theta_aec_dense.npy')

data_alpha_norm = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_alpha_aec_distance_dense.npy')
data_beta_norm = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_beta_aec_distance_dense.npy')

data_hgamma_norm = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_high_gamma1_aec_distance_dense.npy')
data_lgamma_norm = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_low_gamma_aec_distance_dense.npy')
data_theta_norm = np.load(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/{patient}_connectivity_data_theta_aec_distance_dense.npy')
# Reading the .tsv file 
xyz_loc = pd.read_csv(f'/home/pablo/works/dev_thesis_SEEG/outputs_t03/{patient}/ref_0/percentile_0.9/xyz_loc.csv', sep='\t')

# #Data with other metrics
# data_coh = np.load('/home/pablo/works/dev_thesis_SEEG/outputs/pte_01/SR_subseg_connectivity_data_high_freq_coh_dense.npy')[:,:,:,0]
# # Assuming data is the array with shape (50, 127, 127)
# for i in range(data_coh.shape[0]):
#     # Extract the lower triangular part (including the diagonal)
#     lower_triangular = np.tril(data_coh[i])
    
#     # Mirror the lower triangular part to the upper triangular part
#     data_coh[i] = lower_triangular + lower_triangular.T - np.diag(np.diag(lower_triangular))

# #Make data_coh symmetric
# #To know the index of the channels ["lp'11", "lp'12", "op'12", "pi'18", "pa'12"] in the raw object into an array
# idx_channels = [raw.ch_names.index(ch) for ch in ["lp'11", "lp'12", "op'12", "pi'18", "pa'12"]]
# #Eliminate from data_coh the channels that are not in the raw object
# data_coh = np.delete(data_coh, idx_channels, axis=1)
# data_coh = np.delete(data_coh, idx_channels, axis=2)
# # con_data=np.mean(data,axis=3)
# raw.drop_channels(["lp'11", "lp'12", "op'12", "pi'18", "pa'12"])


# inside_channels=['RA01','RA02','RA03','RA04','RB01','RB02','RB03','RB04','RC01','RC02','RC03','RC04','RC05']
inside_channels=['LA01','LA02','LA03','LA04','LA05','LB01','LB02','LB03','LB04']
# #Get the channels outside the SOZ 
# outside_channels=[ch for ch in raw.ch_names if ch not in inside_channels]


In [15]:
data=data_lgamma
data=data.transpose(1,2,0)
percent=0.9

tnet_bu= TemporalNetwork(N=data.shape[0],T=data.shape[2],nettype='wu',from_array=data,
                      timetype='discrete',timeunit='epoch', nodelabels=list(xyz_loc['formatted_label'].values))

tnet_bu.binarize(threshold_type='percent',threshold_level=1-percent,axis='graphlet')

tnet_wu=TemporalNetwork(N=data.shape[0],T=data.shape[2],nettype='wu',from_array=data,
                      timetype='discrete',timeunit='epoch', nodelabels=list(xyz_loc['formatted_label'].values))

tnet_bu_ar=tnet_bu.network
communities_dict={}
k=4
algorithm='k_clique_communities'

for i in range(tnet_bu_ar.shape[2]):
    #Building a network using adjacency matrix
    adj = Adjacency(tnet_bu_ar[:,:,i],labels=raw.ch_names)
    
    G = nx.Graph(adj.to_graph())
    #Community detection algorithm 
    if algorithm=='girvan_newman':
        communities_generator=nx.community.girvan_newman(G)
        next_level_communities = next(communities_generator)
        communities = sorted(map(sorted, next_level_communities))
    elif algorithm=='edge_current_flow_betweenness_partition':
        communities_generator=nx.community.edge_current_flow_betweenness_partition(G, number_of_sets=k)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='edge_betweenness_partition':
        communities_generator=nx.community.edge_betweenness_partition(G, number_of_sets=k)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='k_clique_communities':
        communities_generator=list(nx.community.k_clique_communities(G,k))
        communities=[list(c) for c in communities_generator]
    elif algorithm=='greedy_modularity_communities':
        communities_generator=nx.community.greedy_modularity_communities(G)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='naive_greedy_modularity_communities': 
        communities_generator=nx.community.naive_greedy_modularity_communities(G)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='fast_label_propagation_communities':
        communities_generator=nx.community.fast_label_propagation_communities(G)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='louvain_communities':
        communities_generator=nx.community.louvain_communities(G)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='asyn_fluidc':
        #this algorithm only works for undirected and completely connected graphs
        communities_generator=nx.community.asyn_fluidc(G,k)
        communities=[list(c) for c in communities_generator]
    elif algorithm=='kernighan_lin_bisection':
        communities = [list(nx.community.kernighan_lin_bisection(G)[0])]
        communities.append(list(nx.community.kernighan_lin_bisection(G)[1]))
    communities_dict[i]=communities

# Number of time steps
num_timesteps = len(communities_dict)

# Number of channels
num_channels = len(raw.ch_names)

# Initialize an array for each time step (2D list) with default value -1 (indicating no community yet)
output = [[-1 for _ in range(num_timesteps)] for _ in range(num_channels)]

# Iterate through the communities dictionary
for t, subcommunities in communities_dict.items():
    # Loop through each subcommunity and assign its index to the respective channels
    for subcom_idx, subcommunity in enumerate(subcommunities):
        for channel in subcommunity:
            # Find the index of the channel in ch_names
            if channel in raw.ch_names:
                channel_idx = raw.ch_names.index(channel)
                # Assign the subcommunity index to the correct channel
                output[channel_idx][t] = subcom_idx

#Making temporal consensus
communities_after=teneto.communitydetection.make_temporal_consensus(output)

In [ ]:
#Compare the communities
fig, axes = plt.subplots(1, 2, figsize=(15, 20))

# Plot for 'Community assignment over time before temporal consensus'
im1 = axes[0].imshow(output, aspect='auto', cmap='tab20')
axes[0].set_xlabel('Time step')
axes[0].set_ylabel('Channel index')
axes[0].set_title('Community assignment over time before temporal consensus')
axes[0].set_yticks(np.arange(0, len(raw.ch_names), 1))
axes[0].set_yticklabels(raw.ch_names)
fig.colorbar(im1, ax=axes[0])

# Plot for 'Community assignment over time after temporal consensus'
im2 = axes[1].imshow(communities_after, aspect='auto', cmap='tab20')
axes[1].set_xlabel('Time step')
axes[1].set_ylabel('Channel index')
axes[1].set_title('Community assignment over time after temporal consensus')
axes[1].set_yticks(np.arange(0, len(raw.ch_names), 1))
axes[1].set_yticklabels(raw.ch_names)
fig.colorbar(im2, ax=axes[1])

plt.suptitle(f'Community detection using {algorithm} \n')


plt.tight_layout()
plt.show()


In [ ]:
allegiance_m=teneto.communitymeasures.allegiance(communities_after)
plt.figure(figsize=(20,20))
sns.heatmap(allegiance_m,cmap='viridis')
plt.title('Allegiance')
plt.xlabel('Channels')
plt.ylabel('Communities')
# plt.yticks(np.arange(0, len(xyz_loc['formatted_label'].values), 1), xyz_loc['formatted_label'].values)
# plt.xticks(np.arange(0, len(xyz_loc['formatted_label'].values), 1), xyz_loc['formatted_label'].values)
plt.show()

In [40]:
#Rebuilding the communities for each time step
communities_dict_after={}
for t in range(len(communities_after[0])):
    temp_com={}
    for i, ch in enumerate(raw.ch_names):
        temp_com[communities_after[i][t]]=temp_com.get(communities_after[i][t],[])+[ch]
    communities_dict_after[t]=temp_com       


In [41]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [ ]:
communities_dict_after

In [58]:
#WE are going to extrac the Jaccard index for each community in each time step
#Comparing with the inside channels
jaccard_index={}
for t in range(len(communities_dict_after)):
    temp_jaccard={}
    for com in communities_dict_after[t]:
        temp_jaccard[com]=jaccard_similarity(inside_channels,communities_dict_after[t][com])
    jaccard_index[t]=temp_jaccard

In [59]:
#Get the maximum for each time step, and the community channels that belong to it 
max_jaccard=[]
max_jaccard_com=[]
for t in range(len(jaccard_index)):
    max_jaccard.append(max(jaccard_index[t].values()))
    max_jaccard_com.append(max(jaccard_index[t], key=jaccard_index[t].get))

In [ ]:
#Plotting max_jaccard   
plt.figure(figsize=(10,5))
plt.plot(max_jaccard)
plt.title('Jaccard index')
plt.xlabel('Time step')
plt.ylabel('Jaccard index')
plt.show()

In [ ]:
#printing the communities of maximum jaccard index
dict_communities={}
for t in range(len(communities_dict_after)):
    print(f'Time step {t}: {communities_dict_after[t][max_jaccard_com[t]]}')
    dict_communities[t]=communities_dict_after[t][max_jaccard_com[t]]

print('The inside channels are:',inside_channels)

In [ ]:
## Other similarity metrics
# Overlap 
overlap_index={}
for t in range(len(communities_dict_after)):
    temp_overlap={}
    for com in communities_dict_after[t]:
        temp_overlap[com]=len(set(inside_channels).intersection(communities_dict_after[t][com])) / max(len(inside_channels),len(communities_dict_after[t][com]))
    overlap_index[t]=temp_overlap

#Get the maximum for each time step, and the community channels that belong to it
max_overlap=[]
max_overlap_com=[]
for t in range(len(overlap_index)):
    max_overlap.append(max(overlap_index[t].values()))
    max_overlap_com.append(max(overlap_index[t], key=overlap_index[t].get))

#Plotting max_overlap
plt.figure(figsize=(10,5))
plt.plot(max_overlap)
plt.title('Overlap index')
plt.xlabel('Time step')
plt.ylabel('Overlap index')
plt.show()

#printing the communities of maximum overlap index

for t in range(len(communities_dict_after)):
    print(f'Time step {t}: {communities_dict_after[t][max_overlap_com[t]]}')

print('The inside channels are:',inside_channels)


### For making the method even more unsupervised, we are going to analyze the communities generated at every timestep 

In [32]:
adj = Adjacency(tnet_bu_ar[:,:,2],labels=raw.ch_names)

In [33]:
com_2=communities_dict_after[2]
#Extract the adjacency matrix for each community
adj_com={}
for com, ch in com_2.items():
    com_idx=[raw.ch_names.index(chh) for chh in ch]
    adj_com[com]=Adjacency(tnet_bu_ar[com_idx][:,com_idx,2],labels=ch)

#Make it a graph
G = nx.Graph(adj_com[2].to_graph())
#Compute the eigenvector centrality and the betweenness centrality
eigenvector_centrality = nx.katz_centrality_numpy(G)
betweenness_centrality = nx.betweenness_centrality(G)

In [ ]:
communities_dict_after.keys()

In [ ]:
nx.radius(nx.Graph(adj_com[1].to_graph()))

In [63]:
import networkx as nx

# Function to calculate conductance of a community
def conductance(G, community):
    """
    Calculate the conductance of a single community in a graph G.

    Parameters:
    - G: The graph (networkx.Graph)
    - community: A set of nodes representing the community

    Returns:
    - conductance_score: The conductance of the community
    """
    # Convert the community set to a list
    community = set(community)
    
    # Compute the number of edges between the community and the rest of the graph (cut size)
    cut_size = nx.cut_size(G, community)
    print('The cut size is:',cut_size)
    
    # Compute the volume of the community (sum of degrees of the nodes inside the community)
    community_volume = sum(G.degree(n) for n in community)
    print('The community volume is:',community_volume)
    
    # Compute the volume of the rest of the graph
    outside_volume = sum(G.degree(n) for n in set(G.nodes()) - community)
    print('The outside volume is:',outside_volume)
    
    # Calculate the conductance score
    conductance_score = cut_size / (min(community_volume, outside_volume))
    print('The conductance score is:',conductance_score)
    
    return conductance_score

def normalized_cut(G, community):
    """
    Calculate the normalized cut for a single community in a graph G.

    Parameters:
    - G: The graph (networkx.Graph)
    - community: A set of nodes representing the community

    Returns:
    - normalized_cut_score: The normalized cut value of the community
    """
    # Convert the community set to a set for better performance
    community = set(community)
    
    # Compute the number of edges between the community and the rest of the graph (cut size)
    cut_size = nx.cut_size(G, community)
    
    # Compute the volume of the community (sum of degrees of nodes inside the community)
    community_volume = sum(G.degree(n) for n in community)
    
    # Compute the volume of the rest of the graph (sum of degrees of nodes outside the community)
    outside_community = set(G.nodes()) - community
    outside_volume = sum(G.degree(n) for n in outside_community)
    
    # Avoid division by zero by checking volumes
    if community_volume == 0 or outside_volume == 0:
        return float('inf')  # Return a large value if one of the volumes is zero (invalid cut)
    
    # Calculate normalized cut
    normalized_cut_score = (cut_size / community_volume) + (cut_size / outside_volume)
    
    return normalized_cut_score


import networkx as nx
from itertools import combinations

def intra_community_density_with_size_regularization(G, community, alpha=0.6, size_threshold=15):
    """
    Calculate the intra-community density with size regularization for a community in a graph.

    Parameters:
    - G: The graph (networkx.Graph)
    - community: A set of nodes representing the community
    - alpha: Regularization factor to penalize large or small communities (default: 0.1)
    - size_threshold: The ideal size of the community (default: 10)

    Returns:
    - density_score: The intra-community density with regularization
    """
    community = set(community)
    num_nodes = len(community)
    
    if num_nodes <= 3:
        return 0  # If the community is too small, return density as 0

    # Count edges within the community
    intra_edges = G.subgraph(community).number_of_edges()

    # Calculate intra-community density
    possible_edges = num_nodes * (num_nodes - 1) / 2
    if possible_edges == 0:
        density = 0
    else:
        density = intra_edges / possible_edges
    
    # Size regularization: penalize communities that are too small or too large
    size_penalty = alpha * abs(num_nodes - size_threshold) / size_threshold

    # Regularized density score
    density_score = density - size_penalty
    
    return max(0, density_score)  # Ensure non-negative density score

# t=4
# com_2=communities_dict_after[t]

# # Function to calculate conductance of all communities
# adj = Adjacency(tnet_bu_ar[:,:,t],labels=raw.ch_names)
# G = nx.Graph(adj.to_graph())

# conductance_scores = {}
# for com, ch in com_2.items():
#     # com_idx=[raw.ch_names.index(chh) for chh in ch]
#     # adj_com=Adjacency(tnet_bu_ar[com_idx][:,com_idx,t],labels=ch)
#     # G = nx.Graph(adj_com.to_graph())
#     print('The community is:',ch)
#     conductance_scores[com] = intra_community_density_with_size_regularization(G, ch)

# print(conductance_scores)


#### Building Final communities for ensemble

In [78]:
#Get for every time step the communities that have the maximum intra_community_density_with_size_regularization
max_density_scores=[]
max_density_scores_com=[]
best_communities={}
communities_tuple=[]
for t in range(len(communities_dict_after)):
    com_t=communities_dict_after[t]
    adj = Adjacency(tnet_bu_ar[:,:,t],labels=raw.ch_names)
    G = nx.Graph(adj.to_graph())
    density_scores = {}
    commun=[]
    for com, ch in com_t.items():
        density_scores[com] = intra_community_density_with_size_regularization(G, ch)
        commun.append(ch)
    max_density_scores.append(max(density_scores.values()))
    #Get the community with the maximum density score
    best_communities[t]=commun[list(density_scores.values()).index(max(density_scores.values()))]
    max_density_scores_com.append(max(density_scores, key=density_scores.get))
    communities_tuple.append((best_communities[t],max_density_scores[t]))

In [ ]:
def get_max_communities(data_dict):
    final_array = []

    # Iterate through each time step (t)
    for t in range(len(next(iter(data_dict.values())))):  # Assuming all algorithms have the same time steps
        max_density = float('-inf')
        selected_algorithm = None
        selected_community = None

        # Loop through algorithms to find the max density at time t
        for algorithm, communities in data_dict.items():
            community, density = communities[t]

            # Update the best algorithm-community pair for this time step
            if density > max_density:
                max_density = density
                selected_algorithm = algorithm
                selected_community = community

        # Apply the conditional: if the max density is 0, return an empty community
        if max_density == 0:
            selected_community = []

        # Store the result for this time step
        final_array.append((selected_algorithm, [selected_community], max_density))

    return final_array

result = get_max_communities({'greedy_modularity': communities_tuple})
print(result)

In [67]:
final_communities={}
#Get the t and the communities with a density value different from 0
for t in range(len(best_communities)):
    if max_density_scores[t]!=0:
        final_communities[t]=best_communities[t]

In [ ]:
inside_channels

In [ ]:
final_communities

In [13]:
def jaccard_final_communities(final_communities,comparision_set):
    jaccard_index={}
    for t, com in final_communities.items():
        jaccard_index[t]=jaccard_similarity(comparision_set,com)

    return jaccard_index

# jaccard_final_communities(final_communities,inside_channels)

In [6]:
output_path = '/home/pablo/works/dev_thesis_SEEG/outputs/pte_01/'

with open(output_path+'final_com_high_gamma1_aec_distance_.json') as f:
        communities_data = json.load(f)

In [ ]:
#Getting a plot of the density overtime 
density_scores = [density for _, _, density in communities_data]
plt.figure(figsize=(10,5))
plt.plot(density_scores)
plt.title('Max Intra-community density with size regularization over time')
plt.xlabel('Time step')
plt.ylabel('Density score')
plt.show()

#Getting the Jaccard index for the final communities over time
inside_network = ["m'3","sc'3","sc'4","sc'5","sc'6","y'4","y'5","y'6","y'7","y'8","y'9"]

jaccard=[]
for _, community, _ in communities_data:
    jaccard.append(jaccard_similarity(inside_network,community[0]))

plt.figure(figsize=(10,5))
plt.plot(jaccard)
plt.title('Jaccard index')
plt.xlabel('Time step')
plt.ylabel('Jaccard index')
plt.show()
# communities = [community for _, community, _ in communities_data]
# jaccard_scores = jaccard_final_communities(communities,inside_network)

### Final community

#### Max(max)

In [ ]:
#Reading json file

with open('/home/pablo/works/dev_thesis_SEEG/outputs/sub-HUP142/ref_0/final_com_high_gamma1_aec_.json') as f:
        final_communities_data = json.load(f)

In [ ]:
# Example list
communities = final_communities_data

# "Real contacts" provided in an array
inside_network = set(['LDA1','LDA2','LDA3','LDA4','LDA5','LDA6','LDA7','LDB1','LDB2','LDB3','LDB4','LDB5','LDB6','LDC2','LDC3'])  # Replace with actual data

# Step 1: Assign cumulative densities
contact_densities = {}
for community_type, community_list, density in communities:
    for community in community_list:
        for contact in community:
            if contact not in contact_densities:
                contact_densities[contact] = 0
            contact_densities[contact] += density

# Step 2: Normalize cumulative densities
max_density = max(contact_densities.values())
min_density = min(contact_densities.values())
normalized_densities = {k: (v - min_density) / (max_density - min_density) for k, v in contact_densities.items()}

# Step 3: Prepare for AUC calculation
y_true = np.array([1 if contact in inside_network else 0 for contact in normalized_densities.keys()])
y_scores = np.array([score for score in normalized_densities.values()])

# Step 4: Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
auc_score = auc(fpr, tpr)

# Step 5: Plot the AUC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

#### All densities


In [61]:
#Channels
ref_data=0
patient='sub-HUP144'

main_path='/home/pablo/works/dev_thesis_SEEG/data/mainDatabase_patients/'
doc_file_data=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/'+patient+'_ses-presurgery_scans.tsv',sep='\t')


xyz_loc=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/ieeg/'+patient+'_ses-presurgery_acq-seeg_space-fsaverage_electrodes.tsv',sep='\t')
events=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/'+doc_file_data['filename'][ref_data].replace('_ieeg.edf','_events.tsv'),sep='\t')
channels=pd.read_csv(main_path+'/'+patient+'/'+'ses-presurgery/'+doc_file_data['filename'][ref_data].replace('_ieeg.edf','_channels.tsv'),sep='\t')


good_channels=channels[channels['status']=='good']
inside_network=list(good_channels[(good_channels['status_description'] == 'resect') |
    (good_channels['status_description'] == 'soz') |
    (good_channels['status_description'] == 'resect,soz') |
    (good_channels['status_description'] == 'soz,resect')
]['name'])


with open(f'/home/pablo/works/dev_thesis_SEEG/outputs/{patient}/ref_{ref_data}/Tuples_com_low_gamma_aec_.json') as f:
        final_communities_data = json.load(f)

In [ ]:

# Example list
communities = final_communities_data

# "Real contacts" provided in an 
# Step 1: Assign cumulative densities
contact_densities = {}
for algorithm in communities:
    communities_t=communities[algorithm]
    for community_list, density in communities_t:
        for contact in community_list:
            if contact not in contact_densities:
                contact_densities[contact] = 0
            contact_densities[contact] += density

# Step 2: Normalize cumulative densities
max_density = max(contact_densities.values())
min_density = min(contact_densities.values())
normalized_densities = {k: (v - min_density) / (max_density - min_density) for k, v in contact_densities.items()}

# Step 3: Prepare for AUC calculation
y_true = np.array([1 if contact in inside_network else 0 for contact in normalized_densities.keys()])
y_scores = np.array([score for score in normalized_densities.values()])

# Step 4: Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
auc_score = auc(fpr, tpr)

# Step 5: Plot the AUC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Guess')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
#From dict to pd 
import pandas as pd
df=pd.DataFrame.from_dict(normalized_densities,orient='index',columns=['Normalized density'])
df